# Problem Set 1: Analysis of racial disparities in felony sentencing, Part 1

0. Load packages and imports

In [25]:
import pandas as pd
import numpy as np
import re

## can add others if you need them

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

0.1: Load the data (0 points)

Load the data from `sentencing_asof0405.csv`
- *Notes*: You may receive a warning about mixed data types upon import; feel free to ignore

In [26]:
df = pd.read_csv("C:/Users/sharp/Documents/GitHub/qss20/problemsets/pset1/pset1_inputdata.zip")

C:\Users\sharp\AppData\Local\Temp\ipykernel_24160\1725561910.py:1: DtypeWarning: Columns (10,11,14,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/sharp/Documents/GitHub/qss20/problemsets/pset1/pset1_inputdata.zip")


0.2: Print head, dimensions, info (0 points)

In [27]:
df.head()
df.shape
df.info()


,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,INCIDENT_CITY,INCIDENT_BEGIN_DATE,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY
0,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,False,50510112469,116304211997,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,NaN,8/9/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,8/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,9/21/1984 12:00:00 AM,Homicide
1,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,False,50510213021,98265074680,HOME INVASION,14,12/17/2014 12:00:00 AM,...,NaN,8/9/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,8/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,9/21/1984 12:00:00 AM,Homicide
2,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,False,50516447217,131972895911,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,NaN,8/9/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,8/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,9/21/1984 12:00:00 AM,Homicide
3,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,False,50516497493,131966356472,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,NaN,8/9/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,8/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,9/21/1984 12:00:00 AM,Homicide
4,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,False,50516648320,98059642859,HOME INVASION,13,12/17/2014 12:00:00 AM,...,NaN,8/9/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,8/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,9/21/1984 12:00:00 AM,Homicide


(248146, 41)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248146 entries, 0 to 248145
Data columns (total 41 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   CASE_ID                            248146 non-null  int64  
 1   CASE_PARTICIPANT_ID                248146 non-null  int64  
 2   RECEIVED_DATE                      248146 non-null  object 
 3   OFFENSE_CATEGORY                   248146 non-null  object 
 4   PRIMARY_CHARGE_FLAG                248146 non-null  bool   
 5   CHARGE_ID                          248146 non-null  int64  
 6   CHARGE_VERSION_ID                  248146 non-null  int64  
 7   DISPOSITION_CHARGED_OFFENSE_TITLE  248146 non-null  object 
 8   CHARGE_COUNT                       248146 non-null  int64  
 9   DISPOSITION_DATE                   248146 non-null  object 
 10  DISPOSITION_CHARGED_CHAPTER        248146 non-null  object 
 11  DISPOSITION_CHARGED_ACT            2427

Part 1: data cleaning/interpretation

## 1.1: Understanding the unit of analysis (5 points)

- Print the number of unique values for the following columns. Do so in a way that avoids copying/pasting code for 
the three:

    - Cases (`CASE_ID`)
    - People in that case (`CASE_PARTICIPANT_ID`)
    - Charges (`CHARGE_ID`)

- Write a couple sentences on the following and show an example of each (e.g., a case involving multiple people):
    
    - Why there are more unique people than unique cases?
    - Why there are more unique charges than unique people?

- Print the mean and median number of charges per case

- Print the mean and median number of participants per case

- Does the data seem to enable us to follow the same defendant across different cases they're charged in? Write 1 sentence in support of your conclusion.


In [28]:
# No. of unique values in CASE_ID, CASE_PARTICIPANT_ID, CHARGE_ID, in a way that avoids coping/pasting code for the three
unique_counts = df[["CASE_ID","CASE_PARTICIPANT_ID","CHARGE_ID"]].nunique()
print(unique_counts)

CASE_ID                197519
CASE_PARTICIPANT_ID    211977
CHARGE_ID              229015
dtype: int64


There are more unique people than unique cases because each unique cases can be committed by one or more unique person.

In [29]:
# Find a case with multiple participants
case_counts = df.groupby("CASE_ID")["CASE_PARTICIPANT_ID"].nunique()
multi_participants_ex = case_counts[case_counts > 1].index[0]

# Show the participants for that case
participants = df[df["CASE_ID"] == multi_participants_ex]["CASE_PARTICIPANT_ID"].unique()
print(f"EXAMPLE: CASE_ID {multi_participants_ex} has participants: {participants}")

EXAMPLE: CASE_ID 166402790922 has participants: [144234439761 144234534133]


There are more unique charges than unique people becuase one person can be charged with multiple counts of crime

In [30]:
# Find participants with more than one unique charge
participant_charge_counts = df.groupby("CASE_PARTICIPANT_ID")["CHARGE_ID"].nunique()
multi_charge_participant = participant_charge_counts[participant_charge_counts > 1].index[0]

# Show the charges for that participant
charges = df[df["CASE_PARTICIPANT_ID"] == multi_charge_participant]["DISPOSITION_CHARGED_OFFENSE_TITLE"].unique()
print(f"EXAMPLE: CASE_PARTICIPANT_ID {multi_charge_participant} has charges: {charges}")

EXAMPLE: CASE_PARTICIPANT_ID 97581722610 has charges: ['INT HOMI OF UNBORN CHILD=38-9-1.2' 'MURDER=720-5\\9-1(A)(1-3)'
 'ARMED ROBBERY=720-5\\18-2(A)']


In [31]:
# Mean and median number of charges per case
print(f"Mean number of charges per case: {case_counts.mean():.2f}")
print(f"Median number of charges per case: {case_counts.median():.2f}\n")

# Mean and median number of participants per case
print(f"Mean number of participants per case: {participant_charge_counts.mean():.2f}")
print(f"Median number of participants per case: {participant_charge_counts.median():.2f}")

Mean number of charges per case: 1.07
Median number of charges per case: 1.00

Mean number of participants per case: 1.13
Median number of participants per case: 1.00


The data seem to enable us to follow the same defendant across different cases they're charged in because the CASE_PARTICIPANT_ID is unique to each individual and identifies each defendant across different cases.

## 1.2.1: Which offense is final? (3 points)

- First, read the data documentation [link](https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf) and summarize in your own words the differences between `OFFENSE_CATEGORY` and `UPDATED_OFFENSE_CATEGORY` 

- Construct an indicator `is_changed_offense` that's True for case-participant-charge observations (rows) where there's a difference between the original charge (offense category) and the most current charge (updated offense category). What are some of the more common changed offenses? (can just print result of sort_values based on original offense category)

- Print one example of a changed offense from one of these categories and comment on what the reason may be


OFFENSE_CATEGORY is the case’s initial broad offense label before charges are finalized, while UPDATED_OFFENSE_CATEGORY is the later, revised label recalculated to match the case’s primary/most severe charge.

In [40]:
# Create indicator for changed offense (True if original differs from updated)
is_changed_offense = df["OFFENSE_CATEGORY"] != df["UPDATED_OFFENSE_CATEGORY"]

# Add to datafram
df["is_changed_offense"] = is_changed_offense

# Count the charged offenses based on original offense category
charged_offenses_sorted = df[df["is_changed_offense"]].groupby(df["OFFENSE_CATEGORY"]).size().reset_index(name="count").sort_values("count", ascending=False)

print(f"Total rows with charged offense: {is_changed_offense.sum()}")
print(f"\nMost common charged offenses (by original offense category):")
print(charged_offenses_sorted)

Total rows with charged offense: 35865

Most common charged offenses (by original offense category):
                OFFENSE_CATEGORY  count
61             PROMIS Conversion   6394
33                           DUI   3896
81  UUW - Unlawful Use of Weapon   2155
60                 Other Offense   2125
2             Aggravated Battery   1927
..                           ...    ...
63                       Perjury      4
70                  Prostitution      3
22       Benefit Recipient Fraud      2
29    Compelling Gang Membership      2
85             Violate Bail Bond      2

[88 rows x 2 columns]


In [42]:
# Print one example of a changed offense from a common category
common_offcat_example = df[df["is_changed_offense"]][["OFFENSE_CATEGORY", "UPDATED_OFFENSE_CATEGORY", "CASE_PARTICIPANT_ID", "CHARGE_ID"]].head(1)
print("Example of a changed offense:")
print(common_offcat_example.to_string(index=False))

# Show a more detailed example with the specific change
sample_changed = df[df["is_changed_offense"]].iloc[0]
print(f"\nOriginal offense: {sample_changed['OFFENSE_CATEGORY']}")
print(f"Updated offense: {sample_changed['UPDATED_OFFENSE_CATEGORY']}")

Example of a changed offense:
 OFFENSE_CATEGORY UPDATED_OFFENSE_CATEGORY  CASE_PARTICIPANT_ID   CHARGE_ID
PROMIS Conversion                 Homicide         175691153649 50510112469

Original offense: PROMIS Conversion
Updated offense: Homicide


## 1.2.2: Simplifying the charges (5 points)

Using the field (`UPDATED_OFFENSE_CATEGORY`), create a new field, `simplified_offense_derived`, that simplifies the many offense categories into broader buckets using the following process:

First, combine all offenses beginning with "Aggravated" into a single category without that prefix (e.g., Aggravated Battery and Battery just becomes Battery)

Then:
- Combine all offenses with arson into a single arson category (`Arson`)
- Combine all offenses with homicide into a single homicide category (`Homicide`)
- Combine all offenses with vehicle/vehicular in the name into a single vehicle category (`Vehicle-related`)
- Combine all offenses with battery in the name into a single battery category (`Battery`)

Try to do so efficiently (e.g., write a function and apply to a column, rather than edit the variable repeatedly in separate line for each recoded offense)

Print the difference between the # of unique offenses in the original `UPDATED_OFFENSE_CATEGORY` field and the # of unique offenses in your new `simplified_offense_derived` field


## 1.3: Cleaning additional variables (10 points)

Clean the following variables; make sure to retain the original variable in data and use the derived suffix so it's easier to pull these cleaned out variables later (e.g., `age_derived`) to indicate this was a transformation

- Race: create True/false indicators for `is_black_derived` (Black only or mixed race with hispanic), Non-Black Hispanic, so either hispanic alone or white hispanic (`is_hisp_derived`), White non-hispanic (`is_white_derived`), or none of the above (`is_othereth_derived`)

- Gender: create a boolean true/false indicator for `is_male_derived` (false is female, unknown, or other)

- Age at incident: you notice outliers like 130-year olds. Winsorsize the top 0.01% of values to be equal to the 99.99th percentile value pre-winsorization. Call this `age_derived`

- Create `sentenceymd_derived` that's a version of `SENTENCE_DATE` converted to datetime format. Also create a rounded version, `sentenceym_derived`, that's rounded down to the first day of the month (e.g., `1/5/2016` would become `1/1/2016` and `3/27/2018` would become `3/1/2018`)
    - Hint: all timestamps are midnight so u can strip in conversion. For full credit, before converting, you notice that some of the years have been mistranscribed (e.g., 291X or 221X instead of 201X). Programatically fix those (eg 2914 -> 2014). Even after cleaning, there will still be some that are after the year 2021 that we'll filter out later. For partial credit, you can ignore the timestamps that cause errors and set errors = "coerce" within `pd.to_datetime()` to allow the conversion to proceed. 

- Sentencing judge: create an identifier (`judgeid_derived`) for each unique judge (`SENTENCE_JUDGE`) structured as judge_1, judge_2...., with the order determined by sorting the judges (will sort on fname then last). When finding unique judges, there are various duplicates we could weed out --- for now, just focus on (1) the different iterations of Doug/Douglas Simpson, (2) the different iterations of Shelley Sutker (who appears both with her maiden name and her hyphenated married name). 
     - Hint: due to mixed types, you may need to cast the `SENTENCE_JUDGE` var to a diff type to sort

After finishing, print a random sample of 10 rows (data.sample(n = 10)) with the original and cleaned columns for the relevant variables to validate your work

## 1.4: Subsetting rows to analytic dataset (5 points)

You decide based on the above to simplify things in the following ways:
    
- Subset to cases where only one participant is charged, since cases with >1 participant might have complications like 
plea bargains/informing from other participants affecting the sentencing of the focal participant

- To go from a participant-case level dataset, where each participant is repeated across charges tied to the case, to a participant-level dataset, where each participant has one charge, subset to a participant's primary charge and their current sentence (`PRIMARY_CHARGE_FLAG` is True and `CURRENT_SENTENCE_FLAG` is True). Double check that this worked by confirming there are no longer multiple charges for the same case-participant

- Filter out observations where judge is nan or nonsensical (indicated by is.null or equal to FLOOD)

- Subset to sentencing date between 01-01-2012 and 04-05-2021 (inclusive)

After completing these steps, print the number of rows in the data